In [126]:
import re
import numpy as np
import pandas as pd
import nltk
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score
stopwords = stopwords.words('english')
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

In [107]:
df = pd.read_csv('../data/newtrain.csv')

In [108]:
df.head()

,Category,Text
0,5,why are yawns contagious? when people yawn
1,6,what is trans fat? how to reduce that? i heard...
2,1,roth ira vs 401k? what is the difference betwe...
3,1,how many planes fedex has? i heard that it is ...
4,2,what is the best photo slideshow creation appl...


In [109]:
random_index = np.random.permutation(df.index)

In [110]:
df_shuffled = df.ix[random_index]

In [111]:
df_shuffled.reset_index(drop=True, inplace=True)
df_shuffled[:5]

,Category,Text
0,2,how do i run a ruby program on winxp (i have r...
1,5,what do you call the 4-stringed hawaiian instr...
2,3,when will undertaker of wwe ..........will rea...
3,1,do space heaters really save money? instead of...
4,3,red hot chilli peppers vs foo fighters? whom d...


In [112]:
rows, columns = df_shuffled.shape
train_size = round(rows*.8)
dev_size   = round(rows*.2)

In [115]:
df_train = df_shuffled.loc[:train_size]
df_dev = df_shuffled.loc[train_size:dev_size+train_size].reset_index(drop=True)

In [116]:
df['text_clean'] = df['Text'].apply(lambda x: re.sub('[^A-Za-z0-9]+', ' ', x.lower()))\
                             .apply(lambda x: re.sub('\s+', ' ', x).strip())

In [117]:
def flatten_words(list1d, get_unique=False):
    qa = [s.split() for s in list1d]
    if get_unique:
        return sorted(list(set([w for sent in qa for w in sent])))
    else:
        return [w for sent in qa for w in sent]

In [118]:
words = flatten_words(df.text_clean.values)
words[0:10]

['why',
 'are',
 'yawns',
 'contagious',
 'when',
 'people',
 'yawn',
 'what',
 'is',
 'trans']

In [119]:
def unigram_feature(x, unigram):
    word_list = x.lower().split(" ")
    return word_list.count(unigram)

def bigram_feature(x, bigram):
    bigram_tuple = tuple(bigram.split())
    word_list = x.lower().split(" ")
    bi = nltk.FreqDist(nltk.bigrams(word_list))
    return bi[bigram_tuple]

In [120]:
df_ql = df.copy()
df_ql = df_ql[['Category', 'text_clean']]

In [35]:
df_ql['all_questions'] = df_ql.apply(lambda row:
                                     df.groupby('Category').get_group(row['Category'])['text_clean'].tolist(),
                                     axis=1)

In [37]:
df_ql.drop_duplicates(subset='Category', inplace=True)
df_ql.sort(columns='Category', inplace=True)
df_ql.reset_index(drop=True, inplace=True)

In [65]:
def get_norm_words(collection):
    pattern = r'''(?x)    
         ([A-Z]\.)+        
       | \w+        
       | \$?\d+(\.\d+)?%?  
       | \.\.\.            
     '''
    tokens = nltk.regexp_tokenize(collection, pattern)
    collection_words = [w.lower() for w in tokens if w.lower() not in stopwords and len(w) > 3 ]
    return collection_words

In [82]:
df_ql.all_questions[0]
category1_questions = [q for q in df_ql.all_questions[0]]
category2_questions = [q for q in df_ql.all_questions[1]]
category3_questions = [q for q in df_ql.all_questions[2]]
category4_questions = [q for q in df_ql.all_questions[3]]
category5_questions = [q for q in df_ql.all_questions[4]]
category6_questions = [q for q in df_ql.all_questions[5]]
category7_questions = [q for q in df_ql.all_questions[6]]

In [86]:
category1_words = []
for i in range(len(category1_questions)):
    category1_words += get_norm_words(category1_questions[i])
category2_words = []
for i in range(len(category2_questions)):
    category2_words += get_norm_words(category2_questions[i])
category3_words = []
for i in range(len(category3_questions)):
    category3_words += get_norm_words(category1_questions[i])
category4_words = []
for i in range(len(category4_questions)):
    category4_words += get_norm_words(category4_questions[i])
category5_words = []
for i in range(len(category5_questions)):
    category5_words += get_norm_words(category5_questions[i])
category6_words = []
for i in range(len(category6_questions)):
    category6_words += get_norm_words(category6_questions[i])
category7_words = []
for i in range(len(category7_questions)):
    category7_words += get_norm_words(category7_questions[i])

In [78]:
def norm_unigram_fd(collection_words):
    fd_collection = nltk.FreqDist(collection_words) 
    fd = nltk.FreqDist(words) 
    fd_norm = fd_collection.copy()
    for w in fd_collection.keys():
        fd_norm[w] = (float(fd_collection[w]) / (1+ fd[w]))
    return fd_norm

In [93]:
fd1 = norm_unigram_fd(category1_words)
fd2 = norm_unigram_fd(category2_words)
fd3 = norm_unigram_fd(category3_words)
fd4 = norm_unigram_fd(category4_words)
fd5 = norm_unigram_fd(category5_words)
fd6 = norm_unigram_fd(category6_words)
fd7 = norm_unigram_fd(category7_words)

In [97]:
fd7.max()

'dipole'

In [100]:
train_fd1_feature = df_train['Text'].apply(lambda x: unigram_feature(x, (fd1.max())))
train_fd2_feature = df_train['Text'].apply(lambda x: unigram_feature(x, (fd2.max())))
train_fd3_feature = df_train['Text'].apply(lambda x: unigram_feature(x, (fd3.max())))
train_fd4_feature = df_train['Text'].apply(lambda x: unigram_feature(x, (fd4.max())))
train_fd5_feature = df_train['Text'].apply(lambda x: unigram_feature(x, (fd5.max())))
train_fd6_feature = df_train['Text'].apply(lambda x: unigram_feature(x, (fd6.max())))
train_fd7_feature = df_train['Text'].apply(lambda x: unigram_feature(x, (fd7.max())))

In [102]:
df_train_features = pd.DataFrame({'fd1': train_fd1_feature, 
                                  'fd2': train_fd2_feature, 
                                  'fd3': train_fd3_feature,
                                  'fd4': train_fd4_feature, 
                                  'fd5': train_fd5_feature, 
                                  'fd6': train_fd6_feature,
                                  'fd7': train_fd7_feature})

In [122]:
dev_fd1_feature = df_dev['Text'].apply(lambda x: unigram_feature(x, (fd1.max())))
dev_fd2_feature = df_dev['Text'].apply(lambda x: unigram_feature(x, (fd2.max())))
dev_fd3_feature = df_dev['Text'].apply(lambda x: unigram_feature(x, (fd3.max())))
dev_fd4_feature = df_dev['Text'].apply(lambda x: unigram_feature(x, (fd4.max())))
dev_fd5_feature = df_dev['Text'].apply(lambda x: unigram_feature(x, (fd5.max())))
dev_fd6_feature = df_dev['Text'].apply(lambda x: unigram_feature(x, (fd6.max())))
dev_fd7_feature = df_dev['Text'].apply(lambda x: unigram_feature(x, (fd7.max())))
df_dev_features = pd.DataFrame({'fd1': dev_fd1_feature, 
                                  'fd2': dev_fd2_feature, 
                                  'fd3': dev_fd3_feature,
                                  'fd4': dev_fd4_feature, 
                                  'fd5': dev_fd5_feature, 
                                  'fd6': dev_fd6_feature,
                                  'fd7': dev_fd7_feature})

In [125]:
nb_model = nb.fit(df_train_features, df_train.Category)
nb_predictions = nb_model.predict(df_dev_features)

In [128]:
accuracy_score(df_dev.Category, nb_predictions)

0.28518518518518521